In [ ]:
# 使用lr = 0.01
# 测试xavier_uniform,xavier_norm, kaiming_uniform,kaiming_norm，以及默认的初始化方法 也是一个loop同时还有保存数据部分呢
# 将psnr,mse_gt记录下来即可
# 各跑100遍
import os
current_path = os.getcwd()
print(current_path)
os.chdir("..")
current_path = os.getcwd()
print(current_path)

In [ ]:
import torch
from models.DIP_2D import DIP_2D
import pytorch_lightning as pl
import numpy as np
# 画图相关
import matplotlib.pyplot as plt
# 文件读写相关
import csv
from config.config import *
import pandas as pd
import csv
# 自定义函数
from utils.pre_utils import *
from functools import partial

import datetime

In [ ]:
path_input = "data/noisy_images/uniform_noise3.npy"
path_ground_truth = "data/ground_truth/ground_truth.npy"
path_target = "data/corrupted_images/BSREM_it30.npy"

In [ ]:
image_net_input = np.load(path_input)

# image_net_input_scale,param1_scale_im_net,param2_scale_im_net = rescale_imag(image_net_input,"standardization") 
# image_net_input_torch = torch.Tensor(image_net_input_scale)

image_net_input_torch = torch.Tensor(image_net_input)
image_net_input_torch = image_net_input_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
image_net_input_torch = image_net_input_torch[:,:,:,:,0]

ground_truth = np.load(path_ground_truth)

image_corrupt = np.load(path_target)
image_corrupt_input_scaled,param1_scale_im_corrupt,param2_scale_im_corrupt = rescale_imag(image_corrupt,"standardization") # 标准化图片, 减去平均值，除以标准差，参数1是mean，参数2是std
image_corrupt_torch = torch.Tensor(image_corrupt_input_scaled)
image_corrupt_torch = image_corrupt_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
image_corrupt_torch = image_corrupt_torch[:,:,:,:,0]

plt.imshow(ground_truth, cmap='gray')
plt.show()  

In [ ]:
def write_csv(my_dict, file_name):
    nb_cols = len(list(my_dict.keys()))
    nb_rows = len(my_dict[f"{1}"])
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # 写入表头
        header = ['iters'] + list(range(nb_cols))
        writer.writerow(header)
        
        # 写入数据
        for i in range(nb_rows):
            row = [i] + [my_dict[f"{j}"][i] for j in range(nb_cols)]
            writer.writerow(row)

In [ ]:
# 到100遍
psnr_dict = dict()
mse_gt_dict = dict()
init_dict = dict()
init_method = ['xavier_norm','xavier_uniform','kaiming_norm', 'kaiming_uniform']

# for method in init_method:
#     for i in range(100):
#         # 加载数据
#         train_dataset = torch.utils.data.TensorDataset(image_net_input_torch,image_corrupt_torch)
#         train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 
#         suffix = f'test4/{method}/train_{i}/iters'
#         # 加载模型
#         model = DIP_2D(param1_scale_im_corrupt, param2_scale_im_corrupt, config,'data/Algo/',
#                     "nested",all_images_DIP="False",global_it=-100, suffix=method,last_iter=-1,ground_truth=ground_truth, initial_param=method)
#         model.init_weights()
#         model_class = DIP_2D

#         #定义tensorboard
#         checkpoint_simple_path = os.getcwd() + '/logs/test6_logs'
#         # experiment = 24
#         name=str(datetime.datetime.now())

#         logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path,name=name)#version=format(experiment), name=name)
#         trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=1,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")

#         # 训练模型
#         trainer.fit(model, train_dataloader)
#         out = model(image_net_input_torch)
#         psnr_dict[f"{i}"] = model.psnr_list 
#         mse_gt_dict[f"{i}"] = model.mse_gt_list
        
#         image_out = out.view(PETImage_shape[0],PETImage_shape[1],PETImage_shape[2]).detach().numpy()
#         image_concat = np.concatenate((image_corrupt, destand_numpy_imag(image_out,param1_scale_im_corrupt,param2_scale_im_corrupt)), axis=1)
#         image_reversed = np.max(image_concat)-image_concat
        
#     init_dict[f"{method}"]=psnr_dict,mse_gt_dict  
#     write_csv(psnr_dict, file_name=f"{method}_psnr.csv")
#     write_csv(mse_gt_dict,file_name=f"{method}_mse_gt.csv")

In [ ]:
for i in range(100):
    # 加载数据
    train_dataset = torch.utils.data.TensorDataset(image_net_input_torch,image_corrupt_torch)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 
    suffix = f'test4/default/train_{i}/iters'
    # 加载模型
    model = DIP_2D(param1_scale_im_corrupt, param2_scale_im_corrupt, config,'data/Algo/',
                "nested",all_images_DIP="False",global_it=-100, suffix=suffix,last_iter=-1,ground_truth=ground_truth, initial_param='default')
    #model.init_weights()
    model_class = DIP_2D

    #定义tensorboard
    checkpoint_simple_path = os.getcwd() + '/logs/test6_logs'
    # experiment = 24
    name=str(datetime.datetime.now())

    logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path,name=name)#version=format(experiment), name=name)
    trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=1,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")

    # 训练模型
    trainer.fit(model, train_dataloader)
    out = model(image_net_input_torch)
    psnr_dict[f"{i}"] = model.psnr_list 
    mse_gt_dict[f"{i}"] = model.mse_gt_list
    
    image_out = out.view(PETImage_shape[0],PETImage_shape[1],PETImage_shape[2]).detach().numpy()
    image_concat = np.concatenate((image_corrupt, destand_numpy_imag(image_out,param1_scale_im_corrupt,param2_scale_im_corrupt)), axis=1)
    image_reversed = np.max(image_concat)-image_concat
    
init_dict["default"]=psnr_dict,mse_gt_dict  
write_csv(psnr_dict, file_name="default_psnr.csv")
write_csv(mse_gt_dict,file_name="default_mse_gt.csv")

In [ ]:
# 到100遍
psnr_dict = dict()
mse_gt_dict = dict()
init_dict = dict()
init_method = ['xavier_norm','xavier_uniform','kaiming_norm', 'kaiming_uniform']

for method in init_method:
    for i in range(100):
        # 加载数据
        train_dataset = torch.utils.data.TensorDataset(image_net_input_torch,image_corrupt_torch)
        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 
        suffix = f'test4/{method}/train_{i}/iters'
        # 加载模型
        model = DIP_2D(param1_scale_im_corrupt, param2_scale_im_corrupt, config,'data/Algo/',
                    "nested",all_images_DIP="False",global_it=-100, suffix=suffix,last_iter=-1,ground_truth=ground_truth, initial_param=method)
        model.init_weights()
        model_class = DIP_2D

        #定义tensorboard
        checkpoint_simple_path = os.getcwd() + '/logs/test6_logs'
        # experiment = 24
        name=str(datetime.datetime.now())

        logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path,name=name)#version=format(experiment), name=name)
        trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=1,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")

        # 训练模型
        trainer.fit(model, train_dataloader)
        out = model(image_net_input_torch)
        psnr_dict[f"{i}"] = model.psnr_list 
        mse_gt_dict[f"{i}"] = model.mse_gt_list
        
        image_out = out.view(PETImage_shape[0],PETImage_shape[1],PETImage_shape[2]).detach().numpy()
        image_concat = np.concatenate((image_corrupt, destand_numpy_imag(image_out,param1_scale_im_corrupt,param2_scale_im_corrupt)), axis=1)
        image_reversed = np.max(image_concat)-image_concat
        
    init_dict[f"{method}"]=psnr_dict,mse_gt_dict  
    write_csv(psnr_dict, file_name=f"{method}_psnr.csv")
    write_csv(mse_gt_dict,file_name=f"{method}_mse_gt.csv")

In [ ]:
for i in range(100):
    # 加载数据
    train_dataset = torch.utils.data.TensorDataset(image_net_input_torch,image_corrupt_torch)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 
    suffix = f'test4/default/train_{i}/iters'
    # 加载模型
    model = DIP_2D(param1_scale_im_corrupt, param2_scale_im_corrupt, config,'data/Algo/',
                "nested",all_images_DIP="False",global_it=-100, suffix='default',last_iter=-1,ground_truth=ground_truth, initial_param='default')
    #model.init_weights()
    model_class = DIP_2D

    #定义tensorboard
    checkpoint_simple_path = os.getcwd() + '/logs/test6_logs'
    # experiment = 24
    name=str(datetime.datetime.now())

    logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path,name=name)#version=format(experiment), name=name)
    trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=1,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")

    # 训练模型
    trainer.fit(model, train_dataloader)
    out = model(image_net_input_torch)
    psnr_dict[f"{i}"] = model.psnr_list 
    mse_gt_dict[f"{i}"] = model.mse_gt_list
    
    image_out = out.view(PETImage_shape[0],PETImage_shape[1],PETImage_shape[2]).detach().numpy()
    image_concat = np.concatenate((image_corrupt, destand_numpy_imag(image_out,param1_scale_im_corrupt,param2_scale_im_corrupt)), axis=1)
    image_reversed = np.max(image_concat)-image_concat
    
init_dict["default"]=psnr_dict,mse_gt_dict  
write_csv(psnr_dict, file_name="default_psnr.csv")
write_csv(mse_gt_dict,file_name="default_mse_gt.csv")